In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
path_deg = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/res_deg.csv'
path_mir = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'
path_mir_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results_targets.csv'
path_annotation = '/home/juan/Desktop/juan/bio/mirna_mite/data/introns.gff3'


In [3]:
df_mir = pd.read_csv(path_mir, sep='\t')
df_mir.rename(columns={'MITE':'MITE_prod'}, inplace=True)
total_mir = len(df_mir.index)
print(total_mir)
df_mir[df_mir.MITE_prod.notnull()].head(2)

260


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,22,23,24,seqname,position,start,end,MITE_prod,mirbase,annotation
5,1B:161992200-161992280,sun_all_Cluster_8835,81,135,1.069,5,0.919,+,UCUGUUCACAAAUGUAAGACG,106,...,0,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron
9,1B:565664575-565664794,mrcv_all_Cluster_2513,220,223,1.750,1,0.991,+,CAUUUUCCUAUAGACUUGGUC,142,...,7,0,0,1B,565664575-565664794,565664575,565664794,"MITE_1151,MITE_1572,MITE_1565,MITE_606,MITE_68...",NaN,intron


In [4]:
df_deg = pd.read_csv(path_deg, sep=',')
df_deg.rename(columns={'MITE':'MITE_target'}, inplace=True)
print(len(df_deg.index))
df_deg.head(2)

3128


,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,MITE_target
0,TraesCS1A02G000300.1:66,sun_all_Cluster_118867,TraesCS1A02G000300.1,54,75,66,-43.6,-28.4,0.651376,9.0,"1-2,75-74;5-11,71-65;13-14,63-62;16-22,60-54","3-4,73-72[SIL];12-12,64-64[SIL];15-15,61-61[SIL]",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.951576,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN
1,TraesCS1A02G000400.1:1381,sun_all_Cluster_118867,TraesCS1A02G000400.1,1369,1390,1381,-43.6,-28.4,0.651376,9.0,"1-2,1390-1389;5-11,1386-1380;13-14,1378-1377;1...","3-4,1388-1387[SIL];12-12,1379-1379[SIL];15-15,...",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.955557,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN


In [5]:
has_target = 0
df_mir['has_target'] = 0
has_same_target = 0
for k,v in df_mir.iterrows():
    df_target = df_deg[df_deg.Query == v.Name]
    if len(df_target.index) > 0:
        has_target += 1
        if df_mir.at[k, 'has_target'] == 0:
            df_mir.at[k, 'has_target'] = 1
        if not pd.isna(v.MITE_prod):
            prods = v.MITE_prod.split(',')
            for k2,v2 in df_target.iterrows():
                if not pd.isna(v2.MITE_target):
                    target = v2.MITE_target.split(',')
                    prods = set(filter(None, prods))
                    target = set(filter(None, target))
                    inter = target.intersection(prods)
                    if inter:
                        has_same_target += 1
                        print(v2.SiteID, v.Name, inter)
                        df_mir.at[k, 'has_target'] = 2

TraesCS2D02G327200.1:2349 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G102200.1:1506 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G231700.2:1224 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5B02G480100.1:1530 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5D02G261800.1:1451 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS7A02G218800.1:1394 mrcv_all_Cluster_13743 {'MITE_867'}
TraesCS2A02G274200.2:2583 sun_all_Cluster_136100 {'MITE_376'}
TraesCS2D02G578900.1:1854 sun_all_Cluster_136100 {'MITE_698', 'MITE_376', 'MITE_1128', 'MITE_975', 'MITE_1579'}


In [6]:
total_mir

260

In [7]:
has_target

210

In [8]:
has_target * 100 / total_mir

80.76923076923077

In [9]:
has_same_target

8

In [10]:
len(df_mir[df_mir.has_target == 2].index) * 100 / total_mir

1.1538461538461537

In [11]:
#GENE TARGET REGION

In [12]:
df_ann = pd.read_csv(path_annotation, index_col=False, sep='\t', comment='#', header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print(len(df_ann.index))

2573211


In [13]:
df_ann = df_ann[
    (df_ann.feature == 'intron') | 
    (df_ann.feature == 'exon') | 
    (df_ann.feature == 'three_prime_UTR') | 
    (df_ann.feature == 'five_prime_UTR')]
print(len(df_ann.index))


1593666


In [14]:
df_ann.feature.unique()

array(['exon', 'three_prime_UTR', 'intron', 'five_prime_UTR'],
      dtype=object)

In [16]:
df_ann.tail(2)

,seqname,source,feature,start,end,score,strand,frame,attribute
2573206,Un,Ensembl_Plants,exon,480034703,480034821,.,-,.,Parent=transcript:ENSRNA050014028-T1;Name=ENSR...
2573209,Un,IWGSC,exon,480840068,480840121,.,+,.,Parent=transcript:TraesCSU02G272997.1;Name=Tra...


In [23]:
df_ann['transcript'] = df_ann['attribute'].str.split('transcript:').str[1]
df_ann['transcript'] = df_ann['transcript'].str.split(';').str[0]


In [22]:
df_ann.head(2)

,seqname,source,feature,start,end,score,strand,frame,attribute,transcript
2,1A,Ensembl_Plants,exon,200,1683,.,+,.,Parent=transcript:ENSRNA050013875-T1;Name=ENSR...,:ENSRNA050013875-T1
5,1A,Ensembl_Plants,exon,5023,6833,.,+,.,Parent=transcript:ENSRNA050013847-T1;Name=ENSR...,:ENSRNA050013847-T1


In [ ]:
df_anns = {}
for transcript in df_deg.Transcript.unique():
    df_anns[v.Transcript] = df_ann[df_ann.transcript == v.Transcript]

In [29]:
df_mir['intron'] = 0
df_mir['exon'] = 0
df_mir['three_prime_UTR'] = 0
df_mir['five_prime_UTR'] = 0

In [34]:
for k,v in df_deg.iterrows():
    transcript_start = v.TStart
    transcript_stop = v.TStop
    transcript_name = v.Transcript
    other = df_anns[transcript_name]
    if len(other.index) > 0:
        min_position = min(min(other.start), min(other.end))
        other['start'] -= min_position
        other['end'] -= min_position
        for k2, v2 in other.iterrows():
            if transcript_stop >= v2.start and transcript_start <= v2.end:
                df_mir.loc[df_mir.Name==v.Query, v2.feature] = df_mir[v2.feature].astype(str) + v.Transcript
                print(v.Query)

/home/juan/Desktop/juan/bio/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/juan/Desktop/juan/bio/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


sun_all_Cluster_118867
sun_all_Cluster_118867
sun_all_Cluster_118867
sun_all_Cluster_102510
sun_all_Cluster_118867
sun_all_Cluster_118867
sun_all_Cluster_118867
sun_all_Cluster_86115
sun_all_Cluster_128201
sun_all_Cluster_64707
sun_all_Cluster_118867
sun_all_Cluster_118867
mrcv_all_Cluster_5906
mrcv_all_Cluster_30597
mrcv_all_Cluster_21130
mrcv_all_Cluster_21130
mrcv_all_Cluster_12767
sun_all_Cluster_129257
mrcv_all_Cluster_30239
mrcv_all_Cluster_30239
mrcv_all_Cluster_29262
mrcv_all_Cluster_15025
mrcv_all_Cluster_15025
mrcv_all_Cluster_15025
mrcv_all_Cluster_30597
mrcv_all_Cluster_30597
sun_all_Cluster_19584
sun_all_Cluster_19584
mrcv_all_Cluster_872
mrcv_mites_MITE_T_58879
sun_all_Cluster_19584
sun_all_Cluster_19584
mrcv_mites_MITE_T_121053
mrcv_all_Cluster_13743
mrcv_all_Cluster_12767
mrcv_all_Cluster_12767
mrcv_all_Cluster_12767
sun_all_Cluster_111904
sun_all_Cluster_102510
sun_all_Cluster_56022
sun_all_Cluster_56022
mrcv_mites_MITE_T_40344
mrcv_all_Cluster_7380
mrcv_all_Cluster_73

In [35]:
df_mir.columns

Index(['#Locus', 'Name', 'Length', 'Reads', 'RPM', 'UniqueReads', 'FracTop',
       'Strand', 'MajorRNA', 'MajorRNAReads', 'Complexity', 'DicerCall',
       'MIRNA', 'PhaseScore', 'Short', 'Long', '20', '21', '22', '23', '24',
       'seqname', 'position', 'start', 'end', 'MITE_prod', 'mirbase',
       'annotation', 'has_target', 'intron', 'exon', 'three_prime_UTR',
       'five_prime_UTR'],
      dtype='object')

In [44]:
len(df_mir[df_mir.three_prime_UTR != 0].index)

96

In [45]:
len(df_mir[df_mir.five_prime_UTR != 0].index)

50

In [46]:
len(df_mir[df_mir.exon != 0].index)

192

In [47]:
len(df_mir[df_mir.intron != 0].index)

191

In [48]:
len(df_mir.index)

260

In [49]:
len(df_mir[(df_mir.MITE_prod.notnull())].index)

28

In [51]:
len(df_mir[(df_mir.MITE_prod.notnull()) & (df_mir.three_prime_UTR != 0)].index)

6

In [52]:
df_mir[(df_mir.MITE_prod.notnull()) & (df_mir.three_prime_UTR != 0)]

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,start,end,MITE_prod,mirbase,annotation,has_target,intron,exon,three_prime_UTR,five_prime_UTR
5,1B:161992200-161992280,sun_all_Cluster_8835,81,135,1.069,5,0.919,+,UCUGUUCACAAAUGUAAGACG,106,...,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,2,TraesCS2B02G393000.2TraesCS1A02G138200.2TraesC...,TraesCS2D02G327200.1TraesCS2D02G327200.1TraesC...,TraesCS2D02G327200.1TraesCS2D02G327200.1TraesC...,0
9,1B:565664575-565664794,mrcv_all_Cluster_2513,220,223,1.750,1,0.991,+,CAUUUUCCUAUAGACUUGGUC,142,...,565664575,565664794,"MITE_1151,MITE_1572,MITE_1565,MITE_606,MITE_68...",NaN,intron,1,TraesCS3A02G195900.1TraesCS2A02G234400.1TraesC...,TraesCS2A02G023500.1TraesCS1D02G261600.1TraesC...,0TraesCS4B02G260600.1TraesCS4D02G260400.1Traes...,0
37,2B:482720682-482720772,sun_all_Cluster_33132,91,102,0.808,91,0.078,-,AUCUUCUAUCGUGGGACGAAG,48,...,482720682,482720772,"MITE_1444,MITE_1444,MITE_1030,MITE_1030,DTT_Ta...",NaN,intron,1,TraesCS2A02G317700.3TraesCS2A02G317700.1TraesC...,TraesCS2B02G335700.3TraesCS2B02G335700.3TraesC...,0TraesCSU02G117600.1,0
62,3A:746087222-746087306,mrcv_all_Cluster_12767,85,181,1.421,16,0.000,-,UAAUAUGGAUUGGAGGGGGUA,74,...,746087222,746087306,MITE_660,oan-miR-1400-3p,"three_prime_UTR, exon",1,TraesCS3A02G227900.2TraesCS1A02G033600.1TraesC...,TraesCS3A02G132100.2TraesCS1A02G116200.1TraesC...,TraesCS2D02G102200.1TraesCS1A02G402900.2TraesC...,TraesCS2D02G287000.1TraesCS1B02G068700.1TraesC...
70,3B:587564305-587564383,mrcv_all_Cluster_13743,79,76,0.596,68,1.000,+,AUUGUGUACAGAGGGAGUAGU,67,...,587564305,587564383,MITE_867,NaN,intron,2,TraesCS3A02G122200.4TraesCS1A02G108300.1TraesC...,TraesCS2D02G355700.5TraesCS1A02G419600.1TraesC...,TraesCS1B02G304700.1TraesCS1B02G304700.1TraesC...,0
234,7A:668530089-668530164,sun_all_Cluster_136100,76,96,0.760,2,0.927,+,CUUCUGAUUUACUCGUCGUGG,26,...,668530089,668530164,"MITE_1128,MITE_1128,MITE_975,MITE_975,MITE_698...",NaN,intergenic,2,TraesCS3A02G108900.1TraesCS1A02G247500.1TraesC...,TraesCS3A02G438900.1TraesCS1D02G101600.1TraesC...,TraesCS1D02G101600.1TraesCS1D02G101600.1,0


In [53]:
df_mir[(df_mir.MITE_prod.notnull()) & (df_mir.five_prime_UTR != 0 )]

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,start,end,MITE_prod,mirbase,annotation,has_target,intron,exon,three_prime_UTR,five_prime_UTR
44,2B:720801630-720801713,mrcv_all_Cluster_8534,84,505,3.963,486,1.0,+,AUUGAACUAAGGAGGGGUGGA,386,...,720801630,720801713,"MITE_1643,MITE_1643",NaN,intron,1,TraesCS2D02G521600.1TraesCS1A02G251400.2TraesC...,TraesCS3B02G008600.2TraesCS1A02G310200.1TraesC...,0,0TraesCS7D02G442900.1
62,3A:746087222-746087306,mrcv_all_Cluster_12767,85,181,1.421,16,0.0,-,UAAUAUGGAUUGGAGGGGGUA,74,...,746087222,746087306,MITE_660,oan-miR-1400-3p,"three_prime_UTR, exon",1,TraesCS3A02G227900.2TraesCS1A02G033600.1TraesC...,TraesCS3A02G132100.2TraesCS1A02G116200.1TraesC...,TraesCS2D02G102200.1TraesCS1A02G402900.2TraesC...,TraesCS2D02G287000.1TraesCS1B02G068700.1TraesC...


In [54]:
len(df_mir[(df_mir.MITE_prod.notnull()) & (df_mir.exon != 0 )].index)

10

In [55]:
len(df_mir[(df_mir.MITE_prod.notnull()) & (df_mir.intron !=0)].index)

9

In [ ]:
#add mite TARGET

In [57]:
df_deg.head(2)

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,MITE_target
0,TraesCS1A02G000300.1:66,sun_all_Cluster_118867,TraesCS1A02G000300.1,54,75,66,-43.6,-28.4,0.651376,9.0,"1-2,75-74;5-11,71-65;13-14,63-62;16-22,60-54","3-4,73-72[SIL];12-12,64-64[SIL];15-15,61-61[SIL]",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.951576,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN
1,TraesCS1A02G000400.1:1381,sun_all_Cluster_118867,TraesCS1A02G000400.1,1369,1390,1381,-43.6,-28.4,0.651376,9.0,"1-2,1390-1389;5-11,1386-1380;13-14,1378-1377;1...","3-4,1388-1387[SIL];12-12,1379-1379[SIL];15-15,...",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.955557,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN


In [ ]:
df_deg[['Query','MITE_target']]

In [56]:
df_mir.to_csv(path_mir_res, sep='\t', index=None)